# P-Tuning
1.3B参数模型，全量微调显存25.80G；

在Soft-Prompt-Tuning的基础上，对Prompt进行进一步的编码计算，加速收敛，PEFT中支持LSTM和MLP两种编码方式。

MLP显存6.57G，LSTM显存7.99G（仅针对本次实验的超参数）

## Step1 导入相关包

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "4"

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [2]:
ds = load_dataset("lifefabric/alpaca_data_cleaned.zh", split="train[:10000]")
ds

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10000
})

In [3]:
ds[:3]

{'instruction': ['给出三个保持健康的提示。', '三种主要颜色是什么?', '描述一个原子的结构。'],
 'input': ['', '', ''],
 'output': ['1 。 吃均衡和营养的饮食: 确保你的膳食包括各种水果和蔬菜, 瘦肉蛋白质, 全谷物和健康脂肪。 这有助于为您的身体提供最佳功能的必需营养素, 并有助于预防慢性疾病。\n\n2. 定期进行体育锻炼: 锻炼对于维持强壮的骨骼, 肌肉和心血管健康至关重要。 每周至少进行 150 分钟的中度有氧运动或 75 分钟的剧烈运动。\n\n3. 获得足够的睡眠: 获得足够的高质量睡眠对身心健康至关重要。 它有助于调节情绪, 改善认知功能, 并支持健康的生长和免疫功能。 每天晚上瞄准 7 - 9 小时的睡眠。',
  '三种主要颜色是红色, 蓝色和黄色。 这些颜色被称为主要颜色, 因为它们不能通过混合其他颜色来创建, 所有其他颜色都可以通过以各种比例组合来创建。 在添加色系中, 用于光线的主要颜色是红色, 绿色和蓝色 (R GB) 。',
  '原子是所有物质的基本构建块, 由三种类型的粒子组成: 质子, 中子和电子。 原子的结构可以被描述为中心的一个原子核, 周围是一团电子云。\n\n原子核由质子和中子组成。 质子是带正电的粒子, 中子是没有电荷的中性粒子。 这两个粒子都位于原子核中, 原子核位于原子的中心, 并包含原子的大部分质量。\n\n原子核周围是一团电子云。 电子是在原子核周围不断运动的带负电粒子。 电子云分为壳层或轨道, 每个壳层可以容纳一定数量的电子。 外壳中的电子数, 称为价壳, 决定了原子的化学性质。 \n\n在中性原子中, 原子核中的质子数等于电子云中的电子数, 因此正电荷和负电荷平衡, 原子没有总电荷。 质子的数量, 也称为原子序数, 决定了原子是什么元素。']}

## Step3 数据集预处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [7]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 三种主要颜色是什么?\n\nAssistant: 三种主要颜色是红色, 蓝色和黄色。 这些颜色被称为主要颜色, 因为它们不能通过混合其他颜色来创建, 所有其他颜色都可以通过以各种比例组合来创建。 在添加色系中, 用于光线的主要颜色是红色, 绿色和蓝色 (R GB) 。</s>'

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'三种主要颜色是红色, 蓝色和黄色。 这些颜色被称为主要颜色, 因为它们不能通过混合其他颜色来创建, 所有其他颜色都可以通过以各种比例组合来创建。 在添加色系中, 用于光线的主要颜色是红色, 绿色和蓝色 (R GB) 。</s>'

## Step4 创建模型

In [9]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

In [10]:
sum(param.numel() for param in model.parameters())

1303111680

model size: 1.3B

model: 1.3G * 4 ~= 5.2G

gradient: 1.3G * 4 ~= 5.2G

optimizer: 1.3G * 4 * 2 ~= 10.4G

sum: 20.8G

## P-tuning

### PEFT 1 配置文件

In [11]:
from peft import PromptEncoderConfig, get_peft_model, TaskType, PromptEncoderReparameterizationType

config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM,
                             num_virtual_tokens=10,
                             encoder_reparameterization_type=PromptEncoderReparameterizationType.LSTM,
                             encoder_dropout=0.1,
                             encoder_num_layers=3,
                             encoder_hidden_size=1024)
print(config)

PromptEncoderConfig(peft_type=<PeftType.P_TUNING: 'P_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, encoder_reparameterization_type=<PromptEncoderReparameterizationType.LSTM: 'LSTM'>, encoder_hidden_size=1024, encoder_num_layers=3, encoder_dropout=0.1)


### PEFT 2 创建模型

In [12]:
model = get_peft_model(model, config)

In [13]:
print(config)

PromptEncoderConfig(peft_type=<PeftType.P_TUNING: 'P_TUNING'>, auto_mapping=None, base_model_name_or_path='Langboat/bloom-1b4-zh', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=2048, num_transformer_submodules=1, num_attention_heads=16, num_layers=24, encoder_reparameterization_type=<PromptEncoderReparameterizationType.LSTM: 'LSTM'>, encoder_hidden_size=1024, encoder_num_layers=3, encoder_dropout=0.1)


In [14]:
model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [15]:
model.print_trainable_parameters()

trainable params: 83,959,808 || all params: 1,387,071,488 || trainable%: 6.0530


## Step5 配置训练参数

In [16]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=20,
    num_train_epochs=3
)

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [18]:
trainer.train()

Step,Training Loss
20,3.773100
40,3.103000
60,2.971100
80,2.935100
100,2.796500


KeyboardInterrupt: 

### PEFT 3 加载训练好的peft模型

In [36]:
from peft import PeftModel

peft_model = PeftModel.from_pretrained(model=model, model_id="chatbot/checkpoint-500")

In [45]:
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧可以分为四个部分, 数学思考、阅读、听力和学习如何进行写作。 如果您想获得更好的成绩并参加考试, 请确保这三个步骤同时进行。 为了使阅读和写作变得更容易, 学生可以使用练习阅读, 阅读不同的书籍, 使用不同的写作技巧, 阅读不同的书籍, 并在不同的时间阅读不同类型的东西。'

## Step8 模型推理

In [39]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
pipe.model = peft_model

In [44]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试包括以下几种策略:\n\u3000\u3000 1. 在同一考试中完成更多试题。 有时, 在同一节课上完成更多考试任务会让你更容易集中注意力。 如果你正在准备一个很复杂的练习题, 你可能没有足够的时间完成每个部分, 而您可能需要花更多时间检查每一个步骤。\n\u3000\u3000 2. 你应该花更多的时间复习每个步骤。 复习时间可以帮助你了解自己何时做错了什么。'}]